# Molecular Dynamics Lite workflow
This notebook implements a simple molecular dynamics workflow to demonstrate [Parsl Python parallel scripting](https://parsl-project.org/) in a Jupyter notebook.

## Step 1: Define workflow inputs
This PW workflow can be either launched from its form in the `Compute` tab or it can be run directly in this notebook.  If running directly from the notebook, the user needs to go through the extra step of defining the inputs of the workfow in the notebook.

In [ ]:
import os
from os.path import exists

print('Define workflow inputs...')

# Start assuming workflow is launched from the form.
run_in_notebook=False

if (exists("./params.run")):
    print("Running from a PW form.")
    
else:
    print("Running from a notebook.")
    
    # Set flag for later
    run_in_notebook=True
    
    #TO DO: AUTOMATE THE PROCESS OF GRABING PW.CONF.
    
    # Manually set workflow inputs here
    params="npart;input;25:100:25|steps;input;3000:9000:3000|mass;input;0.01|trsnaps;input;10|"
    
    print(params)
    
    # Write to params.run
    with open("params.run","w") as f:
        n_char_written = f.write(params+"\n")

## Step 2: Configure Parsl
The molecular dynamics software itself is a lightweight, precompiled executable written in C. The executable is distributed with this workflow in `./models/mdlite`, and along with input files, it is staged to the remote resources and does not need to be preinstalled.

The core visualization tool used here is a precompiled binary of [c-ray](https://github.com/vkoskiv/c-ray) distributed with this workflow in `./models/c-ray`. The executable is staged to remote resources and does not need to be preinstalled.

In addition to a Miniconda environment containing Parsl, the only other dependency of this workflow is ImageMagick's `convert` tool for image format conversion (`.ppm` to `.png`) and building animated `.gif` files from `.png` frames.

In [ ]:
print("Configuring Parsl...")
#from glob import glob

import parsl
from parsl.app.app import python_app, bash_app
from parsl.data_provider.files import File
from path import Path
from parslpw import pwconfig,pwargs

if (not run_in_notebook):
    print(pwargs)

parsl.load(pwconfig)
print("pwconfig loaded")

## Step 3: Define Parsl workflow apps
These apps are decorated with Parsl's `@bash_app` and as such are executed in parallel on the compute resources that are defined in the PW configuration loaded above.  Functions that are **not** decorated are not executed in parallel on remote resources. The files that need to be staged to remote resources will be marked with Parsl's `File()` (or its PW extension, `Path()`) in the workflow.

In [ ]:
print("Defining Parsl workflow apps...")

@bash_app
def run_md(stdout='md.app.stdout', stderr='md.app.stderr', inputs=[], outputs=[]):
    return '''
    %s/runMD.sh "%s" trjOut mexOut
    outdir=%s
    mkdir -p $outdir
    mv trjOut $outdir/
    mv mexOut $outdir/
    ''' % (inputs[1],inputs[0],outputs[0])
    
#    import os
#    label = os.path.splitext(os.path.basename(outputs[0]))[0]
#    run_script = os.path.basename(inputs[0])
#    docker = os.path.basename(inputs[2])
#
#    sed_command = "sed -i -e 's,_XYZFILE_,%s.xyz,' -e 's/_NAME_/%s/' %s"%(os.path.splitext(os.path.basename(inputs[1]))[0],label,os.path.basename(inputs[0]))
#    nwchem_command = "/bin/bash " + docker + " " + run_script + " > " + os.path.basename(outputs[0])
#    
#    return '''
#        %s 
#        %s
#        mkdir -p outputs/out
#        mkdir -p outputs/db
#        mv *.db outputs/db
#        mv *.out outputs/out
#    ''' % (sed_command,nwchem_command)

## Step 4: Workflow
This cell executes the workflow itself.

In [ ]:
print("Running workflow...")

# Generate a case list from params.run (the ranges to parameters to sweep)
os.system("python ./models/mexdex/prepinputs.py params.run cases.list")

# Each line in cases.list is a unique combination of the parameters to sweep.
with open("cases.list","r") as f:
    cases_list = f.readlines()

# For each line in cases.list, run and visualize a molecular dynamics simulation
for ii, case in enumerate(cases_list):
    print(case)
    r = run_md(
        inputs=[case,
            Path("./models/mdlite")],
        outputs=[Path("./results/case_"+str(ii))])
    r.result()

# Step 5: Clean up
This step is only necessary when running directly in a notebook. These intermediate and log files are removed to keep the workflow file structure clean if this workflow is pushed into the PW Market Place.  Please feel free to comment out these lines in order to inspect intermediate files as needed. 

In [ ]:
if (run_in_notebook):
    !rm -f params.run
    !rm -rf runinfo
    !rm -rf __pycache__
    !rm -f cases.list
    !rm -rf parsl-task.*